## Installation and Imports


In [2]:
!pip install langchain langchain_community chromadb requests langchain sentence-transformers langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

Now, we can import the required modules:

In [3]:
import os
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain
from langchain.retrievers import BM25Retriever, EnsembleRetriever

## Set up Environment Variables
Before we can use the Hugging Face Hub API, we need to set up the API token as an environment variable. We'll use the os and getpass modules for this purpose.

In [6]:
import os
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass("API:")

# Set the API token in the environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

API:··········


## Load and Split Documents
Here we load the PDF documents from the specified directory and splits them into smaller chunks using the RecursiveCharacterTextSplitter. The chunk size is set to 500 characters with a 50-character overlap.

In [7]:

# Load your documents (assuming they are PDFs in a directory)
loader = PyPDFDirectoryLoader('Data')
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)


## Create Prompt Template:

In [49]:
from langchain.prompts.chat import ChatPromptTemplate

# Define the prompt template as a string
prompt_template = """
You are an AI Assistant that strictly follows instructions.
Your answers must only come from the CONTEXT provided. If the user's query is not related to the CONTEXT, you must respond with "I don't know."

CONTEXT: {context}
</s>

{query}
</s>

Your answer:
"""

# Create a ChatPromptTemplate object directly from the string template
prompt = ChatPromptTemplate.from_template(prompt_template)

## Initialize Embeddings and Vector Store
We initialize the Hugging Face embeddings model and use it to create a Chroma vector store from the document chunks.

In [50]:
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-large")

In [51]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [52]:
vectorstore

## Create BM25 and Vector Retrievers
Here,We create the BM25 and vector retrievers. The BM25 retriever is created directly from the document chunks, while the vector retriever is created from the Chroma vector store.

In [53]:
!pip install rank_bm25

In [54]:
bm25_retriever = BM25Retriever.from_documents(chunks)
vector_retriever = vectorstore.as_retriever()

## Set Up the EnsembleRetriever
 The EnsembleRetriever combines the BM25 and vector retrievers. The weights parameter is set to 0.5 for each retriever, giving them equal importance in the ensemble.

In [55]:
from langchain.retrievers.ensemble import EnsembleRetriever

retrievers = [bm25_retriever, vector_retriever]
ensemble_retriever = EnsembleRetriever(retrievers=retrievers, weights=[0.5, 0.5])


## Initialize the Large Language Model

In [56]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.1,
        "return_full_text":False
    },
)

## Create the RAG Pipeline

In [57]:
from langchain_core.output_parsers import StrOutputParser

In [58]:
from langchain_core.runnables import RunnablePassthrough

In [59]:
output_parser = StrOutputParser()

In [60]:
retriever= ensemble_retriever
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

## Run a Query


In [61]:
query = "what is this story about"


In [62]:
response = chain.invoke(query)

In [63]:
print(response)

Based on the context provided, it is unclear which specific story is being referred to. Please provide more information or clarify which story is being discussed. Without further context, the answer would be "I don't know."


In [66]:
print(chain.invoke("what is electrolysis?"))

Electrolysis is a chemical process that uses electricity to drive nonspontaneous chemical reactions. It involves passing an electric current through an ionic compound (electrolyte) dissolved in a solvent (such as water). This causes the compound to decompose into simpler products, such as metals and gases. Electrolysis has many practical applications, including the production of aluminum, chlorine, and sodium hydroxide. It also plays a crucial role in various industrial processes, such as the refining of metals and the purification of chemicals. In addition, electrolysis is used in electroplating, battery manufacturing, and water treatment. Overall, electrolysis is a versatile and essential technology that has revolutionized numerous industries and continues to advance scientific research today.


In [65]:
print(chain.invoke("what is the publisher's name?"))

I don't know. The context provided does not include information about the publisher's name. Please provide more context or specify which document you are referring to.
